### Parse Swiss data

In [29]:
import requests
from bs4 import BeautifulSoup

cookies = {
    'JSESSIONID': '4E97C0E435F950E02112A7ADA2171F29.wispo2worker',
}

data = [
  ('formular_set', 'formular_set'),
  ('autoScroll', ''),
  ('hiddenActivetabId', '3'),
  ('sort_by', 'LOCATION_ASC'),
  ('wispo_area_search_string', ''),
  ('wispo_area_search_key', ''),
  ('wispo_area_restore_key', ''),
  ('radius_selector', '10'),
  ('check02', 'true'),
  ('search_string', 'Saisir un mot-cl\xE9'),
  ('javax.faces.ViewState', '6281140167725445556:-1337640028376822279'),
  ('javax.faces.source', 'moreResult'),
  ('javax.faces.partial.event', 'click'),
  ('javax.faces.partial.execute', 'moreResult moreResult'),
  ('javax.faces.partial.render', 'main'),
  ('javax.faces.behavior.event', 'action'),
  ('javax.faces.partial.ajax', 'true'),
]

r = requests.post('https://snow.myswitzerland.com/index.jsf', cookies=cookies, data=data)
data = r.text

In [56]:
soup = BeautifulSoup(data, "html5lib")

In [55]:
table = soup.find('table', { "class" : "sportsinfo" })
stations = table.find_all('tr')

190


In [94]:
class Station:
    def __init__(self, name, temp, snow_slopes, snow_station):
        self.name = name
        self.temp = temp
        self.snow_slopes = snow_slopes
        self.snow_station = snow_station
        
    def __str__(self):
        return self.name + " " + self.temp + " " + self.snow_slopes + " " + self.snow_station
    
def extract_station_data(station):
    name = station.find('td', {'class': 'location'}).find('h3').get_text()
    temp = station.find('td', {'class': 'weather info_set1'}).find('em').get_text().strip()
    snow_slopes = station.find('td', {'class': 'info_set2'}).find('em').get_text().strip()
    snow_station = station.find('td', {'class': 'info_set2'}).find('span').get_text().replace('dans la station', '').strip()
    return Station(name, temp, snow_slopes, snow_station)

stations_data = [extract_station_data(station) for station in stations]

In [96]:
[print(station) for station in stations_data]

Adelboden 12° 0 cm 0 cm
Aeschi / Aeschiried 10° 0 cm 0 cm
Airolo 6° n/c 
Aletsch Arena 2° 0 cm 0 cm
Alt St. Johann-Toggenburg 4° 0 cm 0 cm
Amden 12° n/c 
Andermatt-Sedrun SkiArena 2° 35 cm 0 cm
Anzère 10° 0 cm 0 cm
Arolla 2° n/c 
Arosa 2° 0 cm 0 cm
Atzmännig 13° n/c 
Avers 9° 0 cm 0 cm
Balmberg 13° n/c 
Beatenberg 6° n/c 
Belalp Blatten b. Naters 0° 0 cm 0 cm
Bellwald - Top of Goms 6° n/c 
Bergün Filisur 3° n/c 
Bivio 9° n/c 
Bosco Gurin 4° 0 cm 0 cm
Braunwald 9° 0 cm 0 cm
Brienz - Axalp 11° 0 cm 0 cm
Brigels-W'burg-Andiast 12° 0 cm 0 cm
Bruson - Verbier 4° 0 cm 0 cm
Bugnenets - Savagnières  12° 20 cm 10 cm
Bulle - La Chia 11° n/c 
Bumbach / Schangnau 13° n/c 
Campra - Olivone 12° n/c 
Carì 11° n/c 
Celerina/Engadin 1° 0 cm 0 cm
Champex-Lac 12° 0 cm 0 cm
Champéry-Les Crosets PdS 8° 0 cm 0 cm
Chandolin / St-Luc 10° 0 cm 0 cm
Charmey 9° n/c 
Chur - Brambrüesch 6° n/c 
Château-d'Oex - Gstaad 6° 50 cm 10 cm
Chäserrugg - Toggenburg 4° 0 cm 0 cm
Crans - Montana 3° 130 cm 0 cm
Dallenwil Wirzw

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

### Parse data from MagicPass

In [103]:
MP_URL = "https://www.magicpass.ch/fr/stations/"

mp_raw_data = requests.get(MP_URL).text
soup = BeautifulSoup(mp_raw_data, 'html5lib')

mp_stations = soup.find_all('div', {'class': 'station-item'})
mp_stations = [station.find('span', {'class': 'station-name'}).get_text().strip() for station in mp_stations]

In [106]:
print(mp_stations)

['Anzère', 'Bugnenets-Savagnières', 'Charmey', 'Château d’Oex', 'Crans-Montana Aminona', 'Crêt-du-Puy', 'Grimentz-Zinal', 'Jaun', 'La Berra', 'La Lécherette', 'Les Diablerets', 'Les Marécottes', 'Les Mosses', 'Les Paccots', 'Leysin', 'Mayens de Conthey', 'Moléson', 'Nax – Mt-Noble', 'Ovronnaz', 'Rathvel', 'Schwarzsee', 'St-Luc/Chandolin', 'Tramelan', 'Vercorin', 'Villars-Gryon']
